### 일자별 날씨정보 API
-아래사이트의 기상정보를 가져와서 dict 구조로 작성한후 json 파일로 저장하는 내용입니다.
-http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


In [8]:
# !pip install pyOpenSSL
# !pip install request
# !pip install bs4
!pip install tqdm

In [9]:
###################
## 참고: 프로그레스바(진행바)
##################
from tqdm import tqdm

sum=0
for x in tqdm(range(1000)):
    sum+=x


100%|██████████| 1000/1000 [00:00<?, ?it/s]


In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm  #진행사항을 알리는 프로그레스바


In [11]:
########################################
### 전체데이터가 몇개인지 미리 알고 그 갯수를 전체데이터갯수에 넣으면 모든 자료 읽어올수 있음.
#######################################

url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108'
xml=requests.get(url)
soup=bs(xml.text,'html.parser')
totCnt=soup.totalcount.text
totCnt

'365'

In [12]:
# 공공API 인증키 (부성순강사 인증키임. 본인자료는 다시 인증키 받아야함.)
def myUrlOpen(지점코드=108,totCnt=1,startDate='20230101',endDate='20231231'):
    인증키='Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D'
    지점코드=108   #서울
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey='
    url=url+인증키
    url=url+ '&pageNo=1&numOfRows='
    url=url+str(totCnt)+'&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt='
    url=url+startDate+'&endDt='+endDate+'&stnIds='
    url=url+str(지점코드)
    print(url)
    return url


In [13]:
자료=[]
지점리스트={'지점명':['서울','부산','제주'],
              '지점코드': [108,253,184]}
jijumList,dataList,minList,maxList,rnList=[],[],[],[],[]

for 지점명,지점코드 in tqdm(zip(지점리스트['지점명'],지점리스트['지점코드'])):

    url=myUrlOpen(지점코드,totCnt)
    xml=requests.get(url)
    soup=bs(xml.text,'html.parser')
    data=soup.find_all('item')


    for item in data:
        jijum=지점명
        date=item.find('tm').text
        min=item.find('minta').text
        max=item.find('maxta').text
        rn=item.find('sumrn').text

        jijumList.append(jijum) ;dataList.append(date) ; maxList.append(max); minList.append(min) ; rnList.append(rn)

dataList=dict({'j':jijumList,
          'd':dataList,
          'm':maxList,
          'n':minList,
          'r':rnList})

0it [00:00, ?it/s]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


1it [00:01,  1.31s/it]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


2it [00:02,  1.41s/it]

http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=365&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108


3it [00:04,  1.37s/it]


In [14]:
print(지점리스트['지점명'])
fileName='_'.join(지점리스트['지점명'])   # 파일명으로 사용하기 위하여
fileName

['서울', '부산', '제주']


'서울_부산_제주'

In [15]:
#########################3
## 판다스로제작후 저장
#########################
df=pd.DataFrame(dataList)
df.to_csv('C:/Users/blucom005/Downloads/정리폴더/01-03/날씨_'+fileName+'.csv', encoding='utf-8',index=False)
df.tail()

,j,d,m,n,r
1090,제주,2023-12-27,6.8,-2.8,
1091,제주,2023-12-28,4.4,-1.7,
1092,제주,2023-12-29,4.6,-1.5,
1093,제주,2023-12-30,2.1,-0.9,13.1
1094,제주,2023-12-31,4.3,0.6,4.7


In [17]:
################
### json 파일로 저장
##################
import json
with open('C:/Users/blucom005/Downloads/정리폴더/01-03/날씨_'+fileName+'.json', 'w', encoding='utf-8') as f :
	json.dump(dataList, f, indent='\t',ensure_ascii=False)
